In [1]:
import pandas as pd

column_names = ['ano', 'insumos', 'agropecuária', 'industria', 'servicos', 'total']
sheet_cols = ['A:F', 'A,H:L', 'A,N:R', 'A,T:X', 'A,Z:AD', 'A,AF:AJ']
add_column = ['agronegocio', 'agricola', 'pecuario']

list_pib = []

n = 0
for cols in sheet_cols:
    df_pib = pd.read_excel(
        'raw/PIB_Cepea.xlsx',
        header=None,
        usecols=cols,
        skiprows=8,
        nrows=25,
        names=column_names
    )
    df_pib['ramo'] = add_column[n if n < 3 else n - 3]
    df_pib['pib_renda'] = True if n < 3 else False
    list_pib.append(df_pib)
    n += 1

df_pib = pd.concat(list_pib, ignore_index=True)

df_pib.head()

,ano,insumos,agropecuária,industria,servicos,total,ramo,pib_renda
0,1996.0,31597.663029,200211.777672,514755.882623,646536.982537,1.393102e+06,agronegocio,True
1,1997.0,32825.998197,199911.301734,482894.842674,604714.559307,1.320347e+06,agronegocio,True
2,1998.0,31818.410647,200969.741431,455966.002147,576123.413175,1.264878e+06,agronegocio,True
3,1999.0,34784.067160,206952.587184,453199.228768,573119.731296,1.268056e+06,agronegocio,True
4,2000.0,40682.954859,209175.390473,496957.470676,626206.339831,1.373022e+06,agronegocio,True


In [2]:
df_prod = pd.read_excel(
    'raw/produção_2019.xlsx',
    header=None,
    skiprows=6,
    nrows=27,
    names=['uf', 'area_abril', 'area_maio', 'area_junho', 'prod_abril', 'prod_maio', 'prod_junho']
)

df_prod.tail()

,uf,area_abril,area_maio,area_junho,prod_abril,prod_maio,prod_junho
22,RORAIMA,51 827,51 827,51 827,175 314,171 605,171 605
23,SANTA CATARINA,1 284 621,1 281 686,1 281 686,6 540 570,6 540 555,6 540 555
24,SÃO PAULO,2 291 584,2 325 916,2 366 064,9 224 840,9 243 126,10 344 265
25,SERGIPE,147 525,162 722,162 138,695 197,676 097,785 638
26,TOCANTINS,1 368 786,1 427 327,1 427 327,4 374 994,4 779 654,4 779 654


In [3]:
df_prod = df_prod[
    ['area_abril', 'area_maio', 'area_junho', 'prod_abril', 'prod_maio', 'prod_junho']
].applymap(lambda _: _.replace(' ', '')).astype('int').assign(uf=df_prod['uf'])

df_prod.head()

,area_abril,area_maio,area_junho,prod_abril,prod_maio,prod_junho,uf
0,44487,39352,39361,89948,85784,86852,ACRE
1,45842,80283,80385,95917,105458,105525,ALAGOAS
2,21548,22442,23242,50813,60375,62955,AMAPÁ
3,22930,22930,22930,41207,41420,41420,AMAZONAS
4,3130065,3107865,3085865,8283660,9065031,9359331,BAHIA


In [5]:
df_prod_area = df_prod.loc[:, df_prod.columns.str.startswith('area')]\
    .join(df_prod['uf']).melt(id_vars='uf', var_name='mes', value_name='area')

df_prod_area['mes'] = df_prod_area['mes'].apply(lambda _: _.replace('area_', ''))

df_prod_area.sample(4)

,uf,mes,area
41,PARAÍBA,maio,190657
2,AMAPÁ,abril,21548
69,PARANÁ,junho,9800621
62,GOIÁS,junho,5726095


In [6]:
df_prod_prod = df_prod.loc[:, df_prod.columns.str.startswith('prod')]\
    .join(df_prod['uf']).melt(id_vars='uf', var_name='mes', value_name='prod')

df_prod_prod['mes'] = df_prod_prod['mes'].apply(lambda _: _.replace('prod_', ''))

df_prod_prod.query('uf == "ACRE"')

,uf,mes,prod
0,ACRE,abril,89948
27,ACRE,maio,85784
54,ACRE,junho,86852


In [15]:
df_prod_area.join(df_prod_prod)

ValueError: columns overlap but no suffix specified: Index(['uf', 'mes'], dtype='object')

In [8]:
from sqlalchemy import create_engine
from decouple import config

USER_DB = config("USER_DB", default='postgres')
PASSWORD_DB = config("PASSWORD_DB", default='password')
URL_DB = config("URL_DB", default='localhost')
NAME_DB = config("NAME_DB", default='db_name')
PORT_DB = config("PORT_DB", default='5432')
DB_URL = f"postgresql://{USER_DB}:{PASSWORD_DB}@{URL_DB}:{PORT_DB}/{NAME_DB}"


engine = create_engine(DB_URL)

df_pib.to_sql(name='PIP_Cepea', con=engine, if_exists='replace')
df_prod.to_sql(name='producao_2019', con=engine, if_exists='replace')